chmod 600 ~/.ssh/id_ed25519### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Add Binary Class --> Done, Should Add Binary Class Later

4. Summary Statistics Features --> Done

5. Wrapper Functions --> Done, Need to Test Though

6. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

7. Sklearn Pipeline Numerical Features --> StandardScaler Done

8. Create Lagged Features --> Done

9. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

10. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

11. PySpark: XGBoost Classification Feature Importance

In [ ]:
# # Need to Run These in Notebook Version For Pandas UDF
! pip install pyarrow
! pip install pandas
! pip install scikit-learn
! pip install pyspark
! pip install xgboost
! pip install kaleido
! pip install EntropyHub

In [1]:
from Input_Variables.read_vars import train_data_storage, validation_data_storage, test_data_storage, \
                                      inter_train_location, inter_test_location, inter_val_location,\
                                      one_hot_encoding_data, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      model_storage_location, random_seed, \
                                      time_series_lag_values_created, \
                                      evaluation_metrics_output_storage, \
                                      feature_importance_storage_location, \
                                      overall_feature_importance_plot_location

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.imputation_pipeline import Date_And_Value_Imputation


from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Feature_Generation.time_series_feature_creation import TS_Features
from Feature_Generation.difference_features import Difference_Features

from Data_Pipeline.encoding_scaling_pipeline import Feature_Transformations

from Model_Creation.pyspark_xgboost import Create_PySpark_XGBoost

from Model_Predictions.pyspark_model_preds import Model_Predictions

from Model_Evaluation.pyspark_model_eval import Evaluate_Model

from Feature_Importance.model_feature_importance import Feature_Importance

from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

import os

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data()

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Imputation
date_and_value_imputation=Date_And_Value_Imputation()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Complex
ts_features=TS_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# Features Differences
difference_features=Difference_Features()

# PySpark XGBoost Model Module
create_pyspark_xgboost=Create_PySpark_XGBoost()

# Classification Evaluation
evaluate_model=Evaluate_Model()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# Feature Transformations
feature_transformations=Feature_Transformations()


pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()


model_predictions=Model_Predictions()

# Feature Importance
feature_importance=Feature_Importance()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/18 06:19:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# PySpark

### 1. PySpark: Reading In Data

#### Training

In [3]:
training_df=reading_data.read_in_pyspark_data(data_location=train_data_storage)
training_df.show()

+-----+--------------------+-----+-------------------+---------------------+------------------+
|NumId|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+-----+--------------------+-----+-------------------+---------------------+------------------+
| 6563|++3L3PAkDvSTkWnWe...|272.0|2022-02-16 16:10:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|273.0|2022-02-16 16:15:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|270.0|2022-02-16 16:20:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|268.0|2022-02-16 16:25:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|264.0|2022-02-16 16:30:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|262.0|2022-02-16 16:35:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|261.0|2022-02-16 16:40:00|                 null|              null|
| 6563|++3L3PAkDvSTkWnWe...|259.0|2022-0

#### Validation

In [ ]:
validation_df=reading_data.read_in_pyspark_data(data_location=validation_data_storage)
validation_df.show()

#### Testing

In [ ]:
testing_df=reading_data.read_in_pyspark_data(data_location=test_data_storage)
testing_df.show()

### 2. PySpark: Custom Imputation Pipeline

#### Training

In [4]:
interpolation_complete = os.path.exists('/cephfs/interpolation/train')

if interpolation_complete == False:
    date_and_value_imputation.interpolation_creation('train')
    
training_custom_imputation_pipeline = date_and_value_imputation.read_interpolation('/cephfs/interpolation/train/')

training_custom_imputation_pipeline.show(2)

+-------------------+-----+-----+----------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|
+-------------------+-----+-----+----------+
|2022-01-31 18:42:00|    0|153.0|       0.0|
|2022-01-31 18:47:00|    0|156.0|       0.0|
+-------------------+-----+-----+----------+
only showing top 2 rows



#### Testing

In [ ]:
interpolation_complete = os.path.exists('/cephfs/interpolation/test')

if interpolation_complete == False:
    date_and_value_imputation.interpolation_creation('test')
    
testing_custom_imputation_pipeline = date_and_value_imputation.read_interpolation('/cephfs/interpolation/test/')

testing_custom_imputation_pipeline.show(2)

### 3. PySpark: Adding Binary Labels

#### Training

In [5]:
training_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=training_custom_imputation_pipeline)

training_df_added_binary_labels.show(1)

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+-----+-----+----------+--------+--------+--------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|y_Binary|is_above|is_below|
+-------------------+-----+-----+----------+--------+--------+--------+
|2022-01-31 15:19:00|   26|179.0|       0.0|       0|       0|       0|
+-------------------+-----+-----+----------+--------+--------+--------+
only showing top 1 row



#### Testing

In [ ]:
testing_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=testing_custom_imputation_pipeline)

testing_df_added_binary_labels.show(1, truncate=False)

### 4. PySpark: Feature Creation

#### Training

##### Complex Features

In [6]:
training_df_differences = difference_features.add_difference_features(training_custom_imputation_pipeline)
training_df_differences.show(5)

+-------------------+-----+-----+----------+---------+-------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|FirstDiff|SecDiff|
+-------------------+-----+-----+----------+---------+-------+
|2022-01-31 15:19:00|   26|179.0|       0.0|      0.0|    0.0|
|2022-01-31 15:24:00|   26|140.0|       1.0|    -39.0|  -39.0|
|2022-01-31 15:25:00|   26|177.0|       0.0|     37.0|   76.0|
|2022-01-31 15:29:00|   26|177.0|       0.0|      0.0|  -37.0|
|2022-01-31 15:34:00|   26|180.0|       0.0|      3.0|    3.0|
+-------------------+-----+-----+----------+---------+-------+
only showing top 5 rows



In [7]:
training_df_chunks = summary_stats_features.create_chunk_col(training_df_differences, chunk_val = 288)
training_df_chunks.show(5)

+-------------------+-----+-----+----------+---------+-------+-----+-----+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|FirstDiff|SecDiff|index|Chunk|
+-------------------+-----+-----+----------+---------+-------+-----+-----+
|2022-01-31 15:19:00|   26|179.0|       0.0|      0.0|    0.0|    1|    0|
|2022-01-31 15:24:00|   26|140.0|       1.0|    -39.0|  -39.0|    2|    0|
|2022-01-31 15:25:00|   26|177.0|       0.0|     37.0|   76.0|    3|    0|
|2022-01-31 15:29:00|   26|177.0|       0.0|      0.0|  -37.0|    4|    0|
|2022-01-31 15:34:00|   26|180.0|       0.0|      3.0|    3.0|    5|    0|
+-------------------+-----+-----+----------+---------+-------+-----+-----+
only showing top 5 rows



In [8]:
# training_df_poincare = training_df_chunks.groupby(['NumId', 'Chunk']).apply(ts_features.poincare)
# training_df_poincare.show(5)

# training_df_entropy = training_df_chunks.groupby(['NumId', 'Chunk']).apply(ts_features.entropy)
# training_df_entropy.show(5)

+-----+-----+-----------------+----------------+-------------+
|NumId|Chunk|ShortTermVariance|LongTermVariance|VarianceRatio|
+-----+-----+-----------------+----------------+-------------+
|   26|    0|        31.438166|       4.5052047|     6.978188|
|   26|    1|        31.703512|       1.4018252|    22.615881|
|   26|    2|        42.562332|        1.128366|    37.720325|
|   26|    3|        113.79796|       5.9704657|    19.060148|
|   26|    4|        56.827408|       1.9084752|     29.77634|
+-----+-----+-----------------+----------------+-------------+
only showing top 5 rows



+-----+-----+-----------+--------+--------+--------+
|NumId|Chunk|    Entropy|Entropy2|Entropy3|Entropy4|
+-----+-----+-----------+--------+--------+--------+
|   26|    0|  0.3791744|    null|    null|    null|
|   26|    1|  0.1712082|    null|    null|    null|
|   26|    2| 0.24108508|    null|    null|    null|
|   26|    3|0.059399597|    null|    null|    null|
|   26|    4| 0.06988227|    null|    null|    null|
+-----+-----+-----------+--------+--------+--------+
only showing top 5 rows



In [10]:
# training_df_complex_features = training_df_poincare.join(training_df_entropy,['NumId', 'Chunk'])
# training_df_complex_features.show()

/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_

[2796.126s][warning][gc,alloc] Executor task launch worker for task 11.0 in stage 44.0 (TID 2247): Retried waiting for GCLocker too often allocating 524288 words
23/05/18 07:06:27 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.


/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation
/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = short_term_variation / long_term

23/05/18 07:08:18 ERROR Executor: Exception in task 21.0 in stage 44.0 (TID 2257)
org.apache.spark.memory.SparkOutOfMemoryError: Unable to acquire 16384 bytes of memory, got 0
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:158)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:97)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeInMemorySorter.<init>(UnsafeInMemorySorter.java:128)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:165)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:132)
	at org.apache.spark.sql.execution.ExternalAppendOnlyUnsafeRowArray.add(ExternalAppendOnlyUnsafeRowArray.scala:137)
	at org.apache.spark.sql.execution.window.WindowExec$$anon$1.fetchNextPartition(WindowExec.scala:142)
	at org.apache.spark.sql.execution.window.WindowExec$$anon$1.next(WindowExec.scala:174)
	at org.apache.spark.sql.execution.w

23/05/18 07:08:19 WARN TaskSetManager: Lost task 24.0 in stage 44.0 (TID 2260) (jupyter-kkanjaria-40ucsd-2eedu executor driver): TaskKilled (Stage cancelled)
23/05/18 07:08:19 WARN TaskSetManager: Lost task 23.0 in stage 44.0 (TID 2259) (jupyter-kkanjaria-40ucsd-2eedu executor driver): TaskKilled (Stage cancelled)
23/05/18 07:08:19 WARN TaskSetManager: Lost task 22.0 in stage 44.0 (TID 2258) (jupyter-kkanjaria-40ucsd-2eedu executor driver): TaskKilled (Stage cancelled)
23/05/18 07:08:19 WARN TaskSetManager: Lost task 0.0 in stage 45.0 (TID 2261) (jupyter-kkanjaria-40ucsd-2eedu executor driver): TaskKilled (Stage cancelled)


23/05/18 07:08:19 WARN TaskSetManager: Lost task 18.0 in stage 44.0 (TID 2254) (jupyter-kkanjaria-40ucsd-2eedu executor driver): TaskKilled (Stage cancelled)
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/20/temp_shuffle_53da192e-5457-45b4-ad99-addf412d07c2
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/2f/temp_shuffle_b7e841c1-87cd-478b-a0b4-86fa441f331b
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/2e/temp_shuffle_0384e3c0-f664-439b-ba07-9bd4b4cbba28
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/07/temp_shuffle_aba1a57c-db10-47db-8b38-66fdb820eeca
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/1c/temp_shuffle_2ab6a0a1-3908-4c24-9527-c43d41340708
23/05/18 07:08:19

/home/jovyan/glucose-data-analysis/Feature_Generation/time_series_feature_creation.py:55: RuntimeWarning: invalid value encountered in scalar divide
  ratio = short_term_variation / long_term_variation


Py4JJavaError: An error occurred while calling o211.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 21 in stage 44.0 failed 1 times, most recent failure: Lost task 21.0 in stage 44.0 (TID 2257) (jupyter-kkanjaria-40ucsd-2eedu executor driver): org.apache.spark.memory.SparkOutOfMemoryError: Unable to acquire 16384 bytes of memory, got 0
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:158)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:97)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeInMemorySorter.<init>(UnsafeInMemorySorter.java:128)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.<init>(UnsafeExternalSorter.java:165)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.create(UnsafeExternalSorter.java:132)
	at org.apache.spark.sql.execution.ExternalAppendOnlyUnsafeRowArray.add(ExternalAppendOnlyUnsafeRowArray.scala:137)
	at org.apache.spark.sql.e

7:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/0b/temp_shuffle_94c4a5e0-1cf3-4772-91c0-934432f06860
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/1d/temp_shuffle_218f9d2a-2cec-4d0e-8fbe-44a66607b8e9
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/37/temp_shuffle_758e6fdb-d640-491a-89b3-a425816d1040
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/02/temp_shuffle_835b1425-4625-484e-8108-1f2fb5518d4d
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/24/temp_shuffle_743a9bb8-a32e-4ad7-bbeb-fd7d2d7b5176
23/05/18 07:08:19 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-cf5c049c-0b2b-4a9b-9181-8ea0eba743c9/2f/temp_shuffle_981ea4ef-e174-4ef6-8178-57f8eebd67ae
23/05/18 07:08:19 WA

In [ ]:
# training_df_sleep = ts_features.process_for_sleep(df=training_df_differences)
# training_df_sleep.show(5)

##### Statistical Features

In [ ]:
summary_stats_complete = os.path.exists('/cephfs/summary_stats/train')

if summary_stats_complete == False:
    training_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=training_df_chunks)
else:
    training_features_summary_stats=reading_data.read_in_pyspark_data_for_summary_stats('/cephfs/summary_stats/train')

# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

training_features_summary_stats.show(3)

In [ ]:
# #add target variable
# training_features_final_summary = summary_stats_features\
#                                     .add_lag_out_of_range(df=training_features_summary_stats, chunk_lag=1)

# training_features_final_summary.show(4)

In [ ]:
# Merge these together
# training_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

#### Testing

##### Complex Features

In [ ]:
testing_df_differences = difference_features.add_difference_features(testing_df_added_binary_labels)
testing_df_differences.show(5)

In [ ]:
testing_df_chunks = summary_stats_features.create_chunk_col(testing_df_differences, chunk_val = 288)
testing_df_chunks.show(5)

# testing_df_poincare = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.poincare)
# testing_df_poincare.show(5)

# testing_df_entropy = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.entropy)
# testing_df_entropy.show(5)

In [ ]:
# testing_df_complex_features = testing_df_poincare.join(testing_df_entropy,['PatientId', 'Chunk'])
# testing_df_complex_features.show()

In [ ]:
# training_df_sleep = ts_features.process_for_sleep(df=testing_df_added_binary_labels)
# training_df_sleep.show(5)

##### Statistical Features

In [ ]:
testing_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=testing_df_chunks)

# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

testing_features_summary_stats.show(3)

In [ ]:
# Merge these together
# testing_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

### 7. PySpark: Sklearn Regression Pipeline in PySpark

In [ ]:
one_hot_encoded_df=reading_data.read_in_one_hot_encoded_data(one_hot_encoding_location=one_hot_encoding_data)
one_hot_encoded_df=one_hot_encoded_df.select('UserId', 
                                             'Sex_Encoded', 
                                             'Treatment_Encoded', 
                                             'AgeGroup_Encoded')

In [ ]:
one_hot_encoded_df.show(4)

In [ ]:
vecAssembler = VectorAssembler(outputCol="features")
vecAssembler.setInputCols(["Sex_Encoded", "Treatment_Encoded", "AgeGroup_Encoded"])

test = vecAssembler.transform(one_hot_encoded_df)

In [ ]:
test

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(test)

#### Training

In [ ]:
training_encoded=training_features_summary_stats.join(one_hot_encoded_df,
                                                       training_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")


In [ ]:
training_encoded.show(4)

In [ ]:
# merge training_features_summary with 

#### Testing

In [ ]:
testing_encoded=testing_features_summary_stats.join(one_hot_encoded_df,
                                                       testing_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")

In [ ]:
testing_encoded.show(4)

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [ ]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.sql.types import DoubleType, FloatType, LongType
from pyspark.ml import Pipeline

In [ ]:
training_encoded.select('PatientId').show(1, truncate=False)

In [ ]:
patient_1=training_encoded.filter(training_encoded.PatientId=='8W/rpnb48OMm47W2x4FSkc7+9u2mol061DQuJoMdiK0=')

In [ ]:
double_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, LongType)]

# all_numerical=list(set(double_cols+float_cols))
# all_numerical_lags=[x for x in all_numerical if "lag" in x]
all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

# featureArr = [('scaled_' + f) for f in all_numerical_lags]
featureArr = [('scaled_' + f) for f in all_numerical]+['Sex_Encoded', 'Treatment_Encoded', 'AgeGroup_Encoded']
# featureArr = [('scaled_' + f) for f in all_numerical]

va1 = [VectorAssembler(inputCols=[f], outputCol=('vec_' + f)) for f in all_numerical]
ss = [StandardScaler(inputCol='vec_' + f, outputCol='scaled_' + f, withMean=True, withStd=True) for f in all_numerical]

va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

stages = va1 + ss + [va2]

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(training_encoded)

In [ ]:
model.transform(training_encoded).select('features').show(3, truncate=False)

In [ ]:
model.transform(training_encoded).select('features').show(3, truncate=False)

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(patient_1)

In [ ]:
model.transform(patient_1).select('features').show(3, truncate=False)

In [ ]:
# training_numerical_stages=feature_transformations.numerical_scaling(df=training_encoded)
training_numerical_stages=feature_transformations.numerical_scaling(df=training_features_summary_stats)




### 9. PySpark: XGBoost Model

In [ ]:
# xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_encoded,
#                                                         stages=training_numerical_stages,
#                                                         model_storage_location=model_storage_location,
#                                                         random_seed=random_seed)

xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_features_summary_stats,
                                                        stages=training_numerical_stages,
                                                        model_storage_location=model_storage_location,
                                                        random_seed=random_seed)

### 10. PySpark: Cross Validation

### 11. PySpark: Model Predictions

In [ ]:
# testing_predictions=model_predictions.create_predictions_with_model(test_df=testing_encoded, 
#                                                                     model=xgboost_model)
# testing_predictions.show(10)

testing_predictions=model_predictions.create_predictions_with_model(test_df=training_features_summary_stats, 
                                                                    model=xgboost_model)
testing_predictions.show(10)

### 12. PySpark: Model Evaluation

In [ ]:
model_evaluation=evaluate_model.regression_evaluation(testing_predictions=testing_predictions, 
                                                          eval_csv_location=evaluation_metrics_output_storage)

In [ ]:
model_evaluation.head()

### 13. PySpark: XGBoost Classification Feature Importance

In [ ]:
feature_importance_df=feature_importance.\
                        feature_importance_accuracy_gain(xgboost_model=xgboost_model, 
                                                         feature_importance_storage_location=feature_importance_storage_location)


In [ ]:
feature_importance_df.head(10)

### 14. PySpark: Feature Importance Plotting

In [ ]:
overall_feature_plot=xgboost_classification_plot.feature_overall_importance_plot(feature_importance_df=feature_importance_df,
                                                                                 overall_importance_plot_location=overall_feature_importance_plot_location)


In [ ]:
overall_feature_plot.show()

### 15.PySpark: Local Level Feature Importance --> Shap Pandas UDF

In [ ]:
# Add to reqs if this works
! pip install shap

In [ ]:
xgboost_model.stages[-1]

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(xgboost_model.stages[-1])